**Importing Modules**

In [1]:
pip install pyyaml h5py

In [2]:
import os  
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


print(tf.__version__)

2.8.0


Our Dataset

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]


train_images = train_images[:1000].reshape(-1, 28*28)/255.0
test_images = test_images[:1000].reshape(-1, 28*28)/255.0

11501568/11490434 [==============================] - 1s 0us/step


In [4]:
def create_model():
  model = tf.keras.Sequential([
                               keras.layers.Dense(512, activation='relu', input_shape=(784,)),
                               keras.layers.Dropout(0.2),
                               keras.layers.Dense(10)
  ])


  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Checkpoints Callbacks

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]) 

Epoch 1/10
25/32 [======================>.......] - ETA: 0s - loss: 1.2320 - sparse_categorical_accuracy: 0.6550
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 16ms/step - loss: 1.1054 - sparse_categorical_accuracy: 0.6880 - val_loss: 0.7023 - val_sparse_categorical_accuracy: 0.7880
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4091 - sparse_categorical_accuracy: 0.8813
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.4095 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.5078 - val_sparse_categorical_accuracy: 0.8470
Epoch 3/10
30/32 [===========================>..] - ETA: 0s - loss: 0.2678 - sparse_categorical_accuracy: 0.9333
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.2680 - sparse_categorical_accuracy: 0.9330 - val_loss: 0.4464 - val_sparse_categorical_accuracy: 0.8570
Epoch 4/10
27/32 [=========

In [6]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.index', 'cp.ckpt.data-00000-of-00001']

In [7]:
model = create_model()

loss, accu = model.evaluate(test_images,test_labels,verbose=2)

print('Untrain_model, accuracy: {:5.2f}%'.format(100*accu))

32/32 - 0s - loss: 2.4029 - sparse_categorical_accuracy: 0.0760 - 172ms/epoch - 5ms/step
Untrain_model, accuracy:  7.60%


In [8]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


batch_size = 32

cp_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                   verbose=2,
                                   save_weights_only=True,
                                   save_freq = 5*batch_size
                                   )

model = create_model()

model.fit(train_images,train_labels,
          epochs=50,
          validation_data=(test_images,train_labels),
          callbacks = [cp_callback],
          verbose = 0,
          batch_size=batch_size)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [9]:
os.listdir(checkpoint_dir)

['cp-0045.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'checkpoint',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001']

In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [11]:
model = create_model()

model.load_weights(latest)

loss, accu = model.evaluate(test_images,test_labels)
print('Untrain_model, accuracy: {:5.2f}%'.format(100*accu))

32/32 [==============================] - 0s 3ms/step - loss: 0.4769 - sparse_categorical_accuracy: 0.8770
Untrain_model, accuracy: 87.70%


In [12]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4769 - sparse_categorical_accuracy: 0.8770 - 199ms/epoch - 6ms/step
Restored model, accuracy: 87.70%


In [13]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

! mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1653 - sparse_categorical_accuracy: 0.6460
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4462 - sparse_categorical_accuracy: 0.8700
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2795 - sparse_categorical_accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2141 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1543 - sparse_categorical_accuracy: 0.9710
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [18]:
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4297 - sparse_categorical_accuracy: 0.8540 - 214ms/epoch - 7ms/step
Restored model, accuracy: 85.40%
(1000, 10)


In [20]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.1647 - sparse_categorical_accuracy: 0.6480
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4253 - sparse_categorical_accuracy: 0.8850
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2878 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2180 - sparse_categorical_accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1554 - sparse_categorical_accuracy: 0.9690


In [21]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4265 - sparse_categorical_accuracy: 0.8620 - 178ms/epoch - 6ms/step
Restored model, accuracy: 86.20%
